# View OEE Diário

Cálculo de OEE (Overall Equipment Effectiveness) diário.
OEE = Disponibilidade x Performance x Qualidade

In [ ]:
# Configuração (lê parâmetros do job)
try:
    CATALOG = dbutils.widgets.get('catalog')
except:
    CATALOG = 'manufatura_lakehouse'

try:
    SCHEMA_SILVER = dbutils.widgets.get('schema_silver')
except:
    SCHEMA_SILVER = 'silver'

try:
    SCHEMA_GOLD = dbutils.widgets.get('schema_gold')
except:
    SCHEMA_GOLD = 'gold'

def fqtn(schema, table):
    if CATALOG and CATALOG.lower() != 'none':
        return f"`{CATALOG}`.`{schema}`.`{table}`"
    else:
        return f"`{schema}`.`{table}`"

# Criar schema se necessário
if CATALOG and CATALOG.lower() != 'none':
    spark.sql(f"CREATE SCHEMA IF NOT EXISTS `{CATALOG}`.`{SCHEMA_GOLD}`")
    spark.sql(f"USE CATALOG `{CATALOG}`")
else:
    spark.sql(f"CREATE DATABASE IF NOT EXISTS `{SCHEMA_GOLD}`")


In [ ]:

spark.sql(f"""
CREATE OR REPLACE VIEW {fqtn(SCHEMA_GOLD, 'vw_oee_diario')} AS
WITH production_daily AS (
    SELECT 
        equipment_id, 
        start_date_key,
        COALESCE(sum(planned_qty), 0) as total_planned,
        COALESCE(sum(actual_qty), 0) as total_produced,
        -- Trata NULL e valores negativos
        GREATEST(COALESCE(sum(duration_minutes), 0), 0) as production_minutes
    FROM {fqtn(SCHEMA_GOLD, 'fact_producao')}
    WHERE start_date_key IS NOT NULL
    GROUP BY 1, 2
    HAVING GREATEST(COALESCE(sum(duration_minutes), 0), 0) > 0
),
downtime_daily AS (
    SELECT 
        equipment_id, 
        start_date_key,
        COALESCE(sum(downtime_minutes), 0) as total_downtime
    FROM {fqtn(SCHEMA_GOLD, 'fact_manutencao')}
    WHERE start_date_key IS NOT NULL
    GROUP BY 1, 2
),
quality_daily AS (
    SELECT
        equipment_id,
        date_key,
        COALESCE(sum(total_quantity), 0) as total_inspected,
        COALESCE(sum(failed_quantity), 0) as total_failed
    FROM {fqtn(SCHEMA_GOLD, 'fact_qualidade')}
    WHERE date_key IS NOT NULL
    GROUP BY 1, 2
)
SELECT 
    p.start_date_key,
    t.full_date,
    p.equipment_id,
    e.equipment_name,
    COALESCE(d.total_downtime, 0) as downtime_min,
    p.production_minutes,
    (p.production_minutes + COALESCE(d.total_downtime, 0)) as total_time,
    CASE 
        WHEN (p.production_minutes + COALESCE(d.total_downtime, 0)) > 0 
        THEN round(p.production_minutes / (p.production_minutes + COALESCE(d.total_downtime, 0.01)), 4)
        ELSE 0
    END as availability_rate,
    CASE 
        WHEN p.total_planned > 0 
        THEN round(p.total_produced / CAST(p.total_planned AS DOUBLE), 4)
        ELSE 0
    END as performance_rate,
    -- Qualidade: se não há inspeção, assume 100% (1.0)
    CASE 
        WHEN COALESCE(q.total_inspected, 0) > 0 
        THEN round((q.total_inspected - COALESCE(q.total_failed, 0)) / CAST(q.total_inspected AS DOUBLE), 4)
        ELSE 1.0
    END as quality_rate
FROM production_daily p
JOIN {fqtn(SCHEMA_GOLD, 'dim_tempo')} t ON p.start_date_key = t.date_key
JOIN {fqtn(SCHEMA_GOLD, 'dim_equipamento')} e ON p.equipment_id = e.equipment_id
LEFT JOIN downtime_daily d ON p.equipment_id = d.equipment_id AND p.start_date_key = d.start_date_key
LEFT JOIN quality_daily q ON p.equipment_id = q.equipment_id AND p.start_date_key = q.date_key
""")